In [ ]:
import sys
sys.path.insert(0, '../driver')

In [ ]:
import numpy as np
import ephem
import plotly.graph_objects as go
from pyquaternion import Quaternion
from control import polar_rotation_angle, compute_body_trajectory, compute_desired_motor_angles,move_axis_data, MotorSpeedController
from shr import hms2rad, rad2deg, dms2rad, rad2hms, deg2dms, rad2dms

def rms(x):
    return rad2dms(x).replace('d','°')

def dms(x):
    return deg2dms(x).replace('d','°')

In [ ]:
##################################### 
#  PID CONTROL STRATEGY SIMULATION  #
##################################### 
import numpy as np
import plotly.graph_objects as go

def clamp_theta(theta):
    """
    Apply custom bounds to theta[0], theta[1], theta[2]:
    - theta1 ∈ [0, 360]
    - theta2 ∈ [-90, 90]
    - theta3 ∈ [-180, 180]
    """
    clamped = np.empty_like(theta)
    clamped[0] = theta[0] % 360
    clamped[1] = np.clip(theta[1], -90, 90)
    clamped[2] = ((theta[2] + 180) % 360) - 180
    return clamped

def clamp_error(theta_ref, theta_meas):
    """
    Calculates angular error considering wrap-around for each axis.
    Assumes theta1 ∈ [0, 360], theta2 ∈ [-90, 90], theta3 ∈ [-180, 180].
    """
    error = np.empty_like(theta_ref)
    error[0] = ((theta_ref[0] - theta_meas[0] + 180) % 360) - 180
    error[1] = theta_ref[1] - theta_meas[1]
    error[2] = ((theta_ref[2] - theta_meas[2] + 180) % 360) - 180
    return error


def track(dt, theta, omega):
    return clamp_theta(theta + dt * omega)

def predict(dt, theta, omega):
    return clamp_theta(theta + dt * omega)

def errsignal(dt, theta_ref, theta_meas, error_integral):
    error_signal = clamp_error(theta_ref, theta_meas)
    error_integral += error_signal * dt
    return error_signal, error_integral

def pid_control(dt, error_signal, error_integral, omega):
    # Tunable gains
    Kp = 0.8             # Proportional gain base value
    Ki = 0.0             # Intal Gain Base - reduce error when reference is ramping
    Kd = 0.8             # Derivative gain - reduce gain when angular velocity higher (damping)
   
    # Damped control signal
    omega_ctrl = Kp * error_signal - Kd * omega + Ki * error_integral

    return omega_ctrl

def constrain(dt, error_signal, omega_current, omega_target):
    # Limits
    a = 3     # maximum acceleration in degrees per seconds²
    v = 8.0   # maximum velocity in degrees per second
    alpha_min = -a #np.array([-a, -a, -a], dtype=float)
    alpha_max = a #np.array([+a, +a, +a], dtype=float)
    omega_min = np.array([-v, -v, -v], dtype=float)
    omega_max = np.array([+v, +v, +v], dtype=float)

    # Compute constrained acceleration
    delta_omega = omega_target - omega_current
    alpha = delta_omega / dt
    alpha_clipped = np.clip(alpha, alpha_min, alpha_max)

    # Apply alpha, average and clip velocity
    omega_ctrl = omega_current + alpha_clipped * dt
    omega_ctrl = omega_ctrl*0.6 + 0.4*omega_current
    omega_ctrl = np.clip(omega_ctrl, omega_min, omega_max)

    return omega_ctrl


# constrain the output control to valid SLOW or FAST values
def control(dt, controllers, omega_ctrl):
    omega_op = np.array([0,0,0], dtype=float)
    # [0.0, 0.0059018, 0.0175906, 0.0478282, 0.0892742, 0.2079884]
    for axis in range(3):
        omega_op[axis] = omega_ctrl[axis]
        raw = controllers[axis]._model.DPS.toRAW(omega_ctrl[axis])
        if abs(raw) < 5.5:
            raw = round(raw,0)  # no fractional SLOW commands
            omega_op[axis] = controllers[axis]._model.RAW.toDPS(raw)
    return omega_op

dt = 200/1000
n_steps = 800


controllers = [ MotorSpeedController(None, axis, None) for axis in range(3) ]

# Initialization
theta_ref = np.array([10, 20, 2], dtype=float)
omega_ref = np.array([0.0062, 0.0082, 0.018], dtype=float)
theta_meas = np.array([140, 0, 0], dtype=float)
omega_op = np.array([0, 0, 0], dtype=float)
error_integral = np.array([0, 0, 0], dtype=float)

# Logging arrays
theta_log = []
ref_log = []
ctrl_log = []

# Simulation loop
for _ in range(n_steps):
    theta_meas = predict(dt, theta_meas, omega_op)
    theta_ref = track(dt, theta_ref, omega_ref)
    error_signal, error_integral = errsignal(dt, theta_ref, theta_meas, error_integral)
    omega_tgt = pid_control(dt, error_signal, error_integral, omega_op)
    omega_ctrl = constrain(dt, error_signal, omega_op, omega_tgt)
    omega_op = control(dt, controllers, omega_ctrl)

    theta_log.append(theta_meas.copy())
    ref_log.append(theta_ref.copy())
    ctrl_log.append(omega_op.copy())

# Convert logs to arrays
theta_log = np.array(theta_log)
ref_log = np.array(ref_log)
ctrl_log = np.array(ctrl_log)



### 📊 Plotly Visualization: θ₁, θ₂, θ₃

motor_labels = ['θ₁', 'θ₂', 'θ₃']
figs = []
time = np.linspace(0, dt * n_steps, n_steps)

for i in range(3):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=ref_log[:, i], name='Reference', line=dict(color='gray', dash='dash')))
    fig.add_trace(go.Scatter(x=time, y=theta_log[:, i], name='Measured', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=time, y=ctrl_log[:, i], name='Control ω', line=dict(color='green')))
    fig.update_layout(
        title=f'Motor Angle {motor_labels[i]}',
        xaxis_title='Time (seconds)',
        yaxis_title='Degrees',
        template='plotly_white'
    )
    fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Simulated tiny step change sequence around zero
t = np.linspace(0, 5, 500)
step_change = 0.05 * np.sin(2 * np.pi * 1 * t)  # Small oscillation around zero

# Parameters
deadband_thresh = 0.02
hysteresis_thresh = 0.03
alpha = 0.2  # Low-pass filter coefficient

# Memory for control signal
omega_raw = step_change.copy()
omega_deadband = np.zeros_like(t)
omega_hysteresis = np.zeros_like(t)
omega_filtered = np.zeros_like(t)

prev_error = 0.0
prev_output = 0.0

for i in range(len(t)):
    error = omega_raw[i]

    # Apply deadband
    if np.abs(error) < deadband_thresh:
        omega_deadband[i] = 0.0
    else:
        omega_deadband[i] = error

    # Apply hysteresis
    if np.sign(error) != np.sign(prev_error) and np.abs(error) < hysteresis_thresh:
        omega_hysteresis[i] = 0.0
    else:
        omega_hysteresis[i] = omega_deadband[i]

    # Apply low-pass filter (first-order lag)
    omega_filtered[i] = alpha * omega_hysteresis[i] + (1 - alpha) * prev_output

    # Update history
    prev_error = error
    prev_output = omega_filtered[i]

# 📊 Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=omega_raw, mode='lines', name='Raw Signal', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=t, y=omega_deadband, mode='lines', name='Deadband Applied', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=t, y=omega_hysteresis, mode='lines', name='Hysteresis Applied', line=dict(color='green')))
fig.add_trace(go.Scatter(x=t, y=omega_filtered, mode='lines', name='Filtered Output', line=dict(color='blue')))

fig.update_layout(
    title='Control Signal Refinement to Suppress Backlash-Induced Reversals',
    xaxis_title='Time (s)',
    yaxis_title='Angular Velocity Command',
    template='plotly_white'
)

fig.show()

In [ ]:

# Set up observer
observer = ephem.Observer()
observer.lat = '-33.859887'
observer.lon = '151.202177'
observer.lat = '-30.4100413'
observer.lon = '152.6370429'
observer.elevation = 39
observer.date = ephem.now()  # Use current time for observer
observer.epoch = ephem.J2000

# 47 Tuc (J2000)
ra = '0h24m07.23s'
dec = '-72d04m35.0s'

ra0 = hms2rad(ra)
dec0 = dms2rad(dec)

# Assign precessed coordinates
body = ephem.FixedBody()
body._ra=ra0
body._dec=dec0
body._epoch=ephem.J2000  # Use J2000 epoch for RA/Dec
body.compute(observer)

az = rad2deg(body.az)
alt = rad2deg(body.alt) 
pa = polar_rotation_angle(observer.lat, body.az, body.alt)

print(f"Target:        RA = {rad2hms(ra0)},  Dec = {rms(dec0)}")
print(f"Alt/Az/Pra:    Az = {dms(az)},  Alt = {dms(alt)},  Polar Rotation Angle = {dms(pa)}")

In [ ]:
N = 60
Δt = 1

azaltroll_ref = compute_body_trajectory(N, Δt, observer, body, roll=15, is_equatorial_roll=False)
for row in azaltroll_ref:
    print([dms(x) for x in row])

In [ ]:
theta_ref = compute_desired_motor_angles(azaltroll_ref)
for row in theta_ref:
    print([dms(x) for x in row])